<a href="https://colab.research.google.com/github/gasoaress/VGG16_Project/blob/main/pavementRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Preparação dos dados
trdata = ImageDataGenerator(rescale=1./255)  # Normalizando os dados
traindata = trdata.flow_from_directory(directory='/content/Img/Train', target_size=(224,224), class_mode='categorical')
tsdata = ImageDataGenerator(rescale=1./255)  # Normalizando os dados
testdata = tsdata.flow_from_directory(directory='/content/Img/Test', target_size=(224,224), class_mode='categorical')

# Modelo VGG16
VGG = keras.applications.VGG16(input_shape=(224,224,3), include_top=False, weights='imagenet')
VGG.trainable = False

num_classes = len(traindata.class_indices)  # Número de classes

model = Sequential([
    VGG,
    Flatten(),
    Dense(units=256, activation="relu"),
    Dense(units=num_classes, activation="softmax")  # Mudança para softmax para múltiplas classes
])

model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

# Treinamento do modelo
hist = model.fit(
    traindata,
    steps_per_epoch=traindata.samples // traindata.batch_size,
    validation_data=testdata,
    validation_steps=testdata.samples // testdata.batch_size,
    epochs=5
)

model.save('vggclf.h5')

# Verificar as chaves no histórico do treinamento
print(hist.history.keys())

# Visualização do treinamento
import matplotlib.pyplot as plt

plt.plot(hist.history["accuracy"])
if "val_accuracy" in hist.history:
    plt.plot(hist.history["val_accuracy"])
plt.plot(hist.history['loss'])
if "val_loss" in hist.history:
    plt.plot(hist.history['val_loss'])

plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
legend_labels = ["Accuracy", "Loss"]
if "val_accuracy" in hist.history:
    legend_labels.append("Validation Accuracy")
if "val_loss" in hist.history:
    legend_labels.append("Validation Loss")
plt.legend(legend_labels)
plt.show()

# Predição
from keras.preprocessing import image
img = image.load_img("/content/crackedGoogle.jpg", target_size=(224,224))
img = np.asarray(img) / 255.0  # Normalizando a imagem de teste
plt.imshow(img)
img = np.expand_dims(img, axis=0)

from keras.models import load_model
saved_model = load_model("vggclf.h5")
output = saved_model.predict(img)

# Imprimir as probabilidades para cada classe
print("Probabilidades das Classes:", output[0])
predicted_class = np.argmax(output[0])
class_labels = list(traindata.class_indices.keys())
print(f"A imagem pertence à classe: {class_labels[predicted_class]}")


Found 540 images belonging to 3 classes.
Found 540 images belonging to 3 classes.
Epoch 1/5
16/16 [==============================] - 594s 38s/step - loss: 2.7603 - accuracy: 0.6752 - val_loss: 0.5818 - val_accuracy: 0.8613
Epoch 2/5
16/16 [==============================] - ETA: 0s - loss: 0.4327 - accuracy: 0.8740 